<h1><center><b> Acquired Intelligence & Adaptive Behaviour </b></center></h1>
<h2><center><i> Optimisation Algorithm Challenge </i></center></h2>

> _If you find any bugs in this notebook, please contact `kagioulis.efstathios@gmail.com`_

This notebook includes the Genetic Algorithm (GA) Challenge, Part 1 of the assignment. You have to deploy your knowledge of GAs from lab 2 to solve a more complex version of the knapsack problem. The goal of this assignment challenge is to simulate a real-life problem where the optimal solution is not known or easy to compute in a limited amount of time.

You are given a more complex version of the Knapsack problem from lab 2 with **500** items and your goal is to write a GA that will achieve the best possible fitness value with only **2 minutes** of runtime.
When looking for the optimal configurations and hyperparameters of your GA you should always check that your GA satisfies the runtime constraint. In other words make sure your GA implementation does not run more than **2 minutes**.

**You are not allowed to use Genetic or Evolutionary algorithm libraries!!**

The deliverable for this assignment is only this IPython Jypyter notebook. You should use the notebook and edit the given template to provide your solution. Additionally to the code, you may write up to 1000 words within the notebook (in text boxes not in comments) to explain which GA algorithm you chose and why. Use a quantitative investigation to arrive at the best configuration of GA and its hyperparameters. Use figures to help you explain your choice of algorithm and the choice of hyperparameters.

# Marking Criteria
- Technical : The quality of your code and the algorithms you present. Clarity and organisation of code will contribute to the quality of the code.
- The implementation of the following mechanisms is considered essential.
    - mutateechanism
    - Crossover Mechanism
    - Selection Mechanism
    - Fitness Function
- Analysis : What motivated your choice of GA? How did you find the optimal parameters for your GA implementation? Use figures and text to convince the reader of this notebook that you have done a thorough analysis.
- Performance : This just the average and best fitness that your algorithm will accomplish for this task, in other words the average and max fitness within the time limit.
- Research : The extent to which you've gone beyond the lecture material and brought in ideas from the course reading, from other sources and your own ideas. This would ,for example, be experimenting with a fitness function or crossover function or other mechanisms inspired by other research papers.



# Knapsack problem

The knapsack (KP) problem is an example of a combinatorial optimization problem, refer to the [wiki](https://en.wikipedia.org/wiki/Knapsack_problem) for a broader overview. 

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/Knapsack.svg/250px-Knapsack.svg.png" width="200"></center>

It is concerned with a knapsack that has positive integer volume (or capacity) $V$. For instance, the knapsack may be able to store 20 cubic metres. There are $N$ distinct items that may potentially be placed in the knapsack. Item $i$ has a positive integer
volume $Vi$ (e.g this object is 8 cubic metres) and positive integer benefit $Bi$ (e.g how benefical is it to have this object in the knapsack). In the most basic form of the problem we will consider there are only one of each item available (0-1 KP).

The goal is to maximize benefit:

$$
\sum_i^N B_i
$$

Subject to the constraint that:

$$
\big ( \sum_i^N V_i \big) \leq  V
$$

For example suppose we have a knapsack that has a capacity of 20 cubic metres ($V$) and $N=10$ items of different sizes and different benefits. We want to include in the knapsack only these
items that will have the greatest total benefit within the constraint of the knapsack’s capacity.

```
item        a b c d e f g  h i j ...
Benefit (B) 5 6 1 9 2 8 4  3 7 10 ...
Volume (V)  3 2 4 5 8 9 10 1 6 7 ...
```

The above is just a reiteration of the definition of the kanpasck problem. For the purposes of this challenge the number of items is *500* and the volume limit is set to a percetage of the total volume of the items. 

Bellow is a example defintion of the 500 item problem.


In [24]:
import numpy as np
import copy
import random
import sys
import operator


max_volume = 5476
benefits = np.array([16.206994374857622, 16.206994374857622, 56.53317610041027, 49.4907398745799, 51.22499389946279, 32.87011476165613, 54.30776494511014, 53.82688299849187, 9.404490653110589, 55.67764362830022, 26.12789058968723, 52.82676089508675, 47.609522856952324, 51.22499389946279, 37.18422604635824, 29.05932629027116, 50.08437325598119, 37.18422604635824, 54.30776494511014, 41.71863425909871, 52.82676089508675, 56.94441734416699, 54.776312804390585, 29.05932629027116, 42.536259042531384, 48.25395781119352, 46.946778377222, 54.30776494511014, 26.12789058968723, 29.05932629027116, 57.344960061407704, 32.87011476165613, 50.08437325598119, 54.776312804390585, 27.640549922170507, 26.12789058968723, 44.840457922133965, 40.87378948258488, 37.18422604635824, 9.404490653110589, 31.66491223210111, 30.397368307141324, 34.01960219246153, 32.87011476165613, 46.946778377222, 45.56314299957806, 46.946778377222, 53.82688299849187, 30.397368307141324, 9.404490653110589, 51.22499389946279, 35.11884584284246, 13.2664991614216, 40.87378948258488, 49.4907398745799, 38.157568056677825, 56.11100110000217, 26.12789058968723, 20.816659994661325, 46.26493752772659, 46.26493752772659, 29.05932629027116, 22.744962812309304, 54.30776494511014, 34.01960219246153, 44.09585518440984, 44.840457922133965, 54.30776494511014, 34.01960219246153, 41.71863425909871, 13.2664991614216, 38.157568056677825, 46.26493752772659, 9.404490653110589, 51.773008840943795, 40.87378948258488, 27.640549922170507, 51.22499389946279, 31.66491223210111, 52.82676089508675, 13.2664991614216, 31.66491223210111, 16.206994374857622, 27.640549922170507, 52.306787322488084, 40.0, 40.87378948258488, 40.87378948258488, 53.33333333333333, 48.25395781119352, 36.172426576668094, 48.25395781119352, 9.404490653110589, 47.609522856952324, 32.87011476165613, 51.22499389946279, 36.172426576668094, 54.776312804390585, 32.87011476165613, 46.946778377222, 52.82676089508675, 40.0, 40.0, 36.172426576668094, 40.0, 43.32820482472512, 47.609522856952324, 53.33333333333333, 18.666666666666664, 50.66228051190221, 36.172426576668094, 40.87378948258488, 39.09532509705533, 26.12789058968723, 32.87011476165613, 53.82688299849187, 47.609522856952324, 43.32820482472512, 48.25395781119352, 56.11100110000217, 56.53317610041027, 32.87011476165613, 24.503967932652138, 18.666666666666664, 30.397368307141324, 55.67764362830022, 48.88080741286229, 45.56314299957806, 27.640549922170507, 24.503967932652138, 51.22499389946279, 44.840457922133965, 54.30776494511014, 35.11884584284246, 55.67764362830022, 52.82676089508675, 52.306787322488084, 42.536259042531384, 16.206994374857622, 9.404490653110589, 30.397368307141324, 46.946778377222, 32.87011476165613, 53.82688299849187, 31.66491223210111, 13.2664991614216, 22.744962812309304, 39.09532509705533, 26.12789058968723, 57.344960061407704, 56.11100110000217, 56.11100110000217, 54.30776494511014, 27.640549922170507, 52.82676089508675, 34.01960219246153, 56.94441734416699, 50.08437325598119, 50.08437325598119, 50.66228051190221, 39.09532509705533, 44.09585518440984, 24.503967932652138, 42.536259042531384, 52.82676089508675, 24.503967932652138, 34.01960219246153, 34.01960219246153, 48.88080741286229, 52.306787322488084, 13.2664991614216, 50.08437325598119, 38.157568056677825, 37.18422604635824, 41.71863425909871, 46.946778377222, 56.94441734416699, 55.67764362830022, 22.744962812309304, 29.05932629027116, 46.26493752772659, 53.82688299849187, 32.87011476165613, 13.2664991614216, 42.536259042531384, 57.344960061407704, 51.22499389946279, 49.4907398745799, 48.25395781119352, 46.26493752772659, 37.18422604635824, 44.09585518440984, 40.0, 13.2664991614216, 29.05932629027116, 43.32820482472512, 13.2664991614216, 40.87378948258488, 42.536259042531384, 46.946778377222, 44.840457922133965, 57.344960061407704, 44.09585518440984, 29.05932629027116, 9.404490653110589, 26.12789058968723, 22.744962812309304, 52.306787322488084, 47.609522856952324, 31.66491223210111, 45.56314299957806, 35.11884584284246, 40.0, 51.773008840943795, 48.88080741286229, 44.09585518440984, 42.536259042531384, 56.94441734416699, 49.4907398745799, 55.67764362830022, 47.609522856952324, 51.773008840943795, 49.4907398745799, 42.536259042531384, 9.404490653110589, 18.666666666666664, 26.12789058968723, 55.67764362830022, 40.87378948258488, 46.946778377222, 41.71863425909871, 42.536259042531384, 56.94441734416699, 47.609522856952324, 35.11884584284246, 54.776312804390585, 32.87011476165613, 36.172426576668094, 51.22499389946279, 52.82676089508675, 16.206994374857622, 46.26493752772659, 26.12789058968723, 26.12789058968723, 9.404490653110589, 46.946778377222, 38.157568056677825, 22.744962812309304, 40.0, 56.53317610041027, 55.232840472554614, 26.12789058968723, 56.11100110000217, 30.397368307141324, 48.88080741286229, 56.53317610041027, 26.12789058968723, 56.11100110000217, 57.344960061407704, 47.609522856952324, 56.94441734416699, 56.94441734416699, 52.82676089508675, 45.56314299957806, 16.206994374857622, 56.94441734416699, 52.82676089508675, 49.4907398745799, 43.32820482472512, 39.09532509705533, 45.56314299957806, 26.12789058968723, 47.609522856952324, 44.09585518440984, 37.18422604635824, 40.0, 43.32820482472512, 47.609522856952324, 29.05932629027116, 18.666666666666664, 24.503967932652138, 22.744962812309304, 54.30776494511014, 54.30776494511014, 48.25395781119352, 43.32820482472512, 45.56314299957806, 36.172426576668094, 26.12789058968723, 49.4907398745799, 48.88080741286229, 52.306787322488084, 32.87011476165613, 44.840457922133965, 41.71863425909871, 54.776312804390585, 50.08437325598119, 30.397368307141324, 48.88080741286229, 48.25395781119352, 9.404490653110589, 46.26493752772659, 55.232840472554614, 26.12789058968723, 44.840457922133965, 55.232840472554614, 42.536259042531384, 52.306787322488084, 52.82676089508675, 29.05932629027116, 45.56314299957806, 51.773008840943795, 13.2664991614216, 16.206994374857622, 27.640549922170507, 54.30776494511014, 56.11100110000217, 18.666666666666664, 43.32820482472512, 22.744962812309304, 39.09532509705533, 42.536259042531384, 44.09585518440984, 42.536259042531384, 48.88080741286229, 50.08437325598119, 50.66228051190221, 22.744962812309304, 54.776312804390585, 46.946778377222, 54.30776494511014, 53.82688299849187, 40.87378948258488, 39.09532509705533, 51.773008840943795, 42.536259042531384, 9.404490653110589, 55.67764362830022, 53.33333333333333, 57.344960061407704, 49.4907398745799, 39.09532509705533, 13.2664991614216, 57.344960061407704, 56.11100110000217, 16.206994374857622, 48.88080741286229, 24.503967932652138, 56.11100110000217, 48.25395781119352, 43.32820482472512, 55.232840472554614, 46.946778377222, 34.01960219246153, 56.53317610041027, 9.404490653110589, 49.4907398745799, 36.172426576668094, 32.87011476165613, 22.744962812309304, 9.404490653110589, 18.666666666666664, 29.05932629027116, 52.82676089508675, 41.71863425909871, 46.26493752772659, 20.816659994661325, 9.404490653110589, 32.87011476165613, 46.26493752772659, 54.30776494511014, 43.32820482472512, 54.776312804390585, 20.816659994661325, 13.2664991614216, 32.87011476165613, 41.71863425909871, 29.05932629027116, 18.666666666666664, 48.88080741286229, 31.66491223210111, 56.11100110000217, 13.2664991614216, 24.503967932652138, 9.404490653110589, 52.306787322488084, 50.08437325598119, 51.773008840943795, 56.11100110000217, 38.157568056677825, 32.87011476165613, 50.08437325598119, 55.232840472554614, 42.536259042531384, 32.87011476165613, 13.2664991614216, 56.11100110000217, 49.4907398745799, 53.82688299849187, 35.11884584284246, 54.776312804390585, 47.609522856952324, 51.22499389946279, 43.32820482472512, 30.397368307141324, 18.666666666666664, 9.404490653110589, 56.11100110000217, 48.25395781119352, 32.87011476165613, 48.88080741286229, 31.66491223210111, 52.82676089508675, 52.82676089508675, 40.87378948258488, 31.66491223210111, 47.609522856952324, 42.536259042531384, 22.744962812309304, 56.53317610041027, 50.08437325598119, 50.66228051190221, 54.776312804390585, 42.536259042531384, 20.816659994661325, 48.25395781119352, 26.12789058968723, 46.26493752772659, 36.172426576668094, 56.53317610041027, 54.776312804390585, 41.71863425909871, 37.18422604635824, 24.503967932652138, 46.946778377222, 52.82676089508675, 16.206994374857622, 42.536259042531384, 40.0, 56.11100110000217, 31.66491223210111, 40.0, 54.30776494511014, 57.344960061407704, 54.30776494511014, 44.840457922133965, 22.744962812309304, 9.404490653110589, 18.666666666666664, 36.172426576668094, 35.11884584284246, 32.87011476165613, 13.2664991614216, 32.87011476165613, 54.30776494511014, 52.82676089508675, 56.94441734416699, 35.11884584284246, 56.11100110000217, 56.94441734416699, 50.08437325598119, 39.09532509705533, 38.157568056677825, 46.946778377222, 48.88080741286229, 24.503967932652138, 54.776312804390585, 18.666666666666664, 20.816659994661325, 46.26493752772659, 55.232840472554614, 48.25395781119352, 40.0, 56.11100110000217, 30.397368307141324, 9.404490653110589, 56.53317610041027, 51.773008840943795, 13.2664991614216, 50.08437325598119, 24.503967932652138, 31.66491223210111, 26.12789058968723, 46.26493752772659, 55.232840472554614, 55.67764362830022, 53.82688299849187, 39.09532509705533, 39.09532509705533, 49.4907398745799, 26.12789058968723, 54.30776494511014, 44.840457922133965, 50.08437325598119, 44.09585518440984, 57.344960061407704, 20.816659994661325, 54.776312804390585, 9.404490653110589, 45.56314299957806])
volumes = np.array([3, 3, 47, 33, 36, 13, 42, 41, 1, 45, 8, 39, 30, 36, 17, 10, 34, 17, 42, 22, 39, 48, 43, 10, 23, 31, 29, 42, 8, 10, 49, 13, 34, 43, 9, 8, 26, 21, 17, 1, 12, 11, 14, 13, 29, 27, 29, 41, 11, 1, 36, 15, 2, 21, 33, 18, 46, 8, 5, 28, 28, 10, 6, 42, 14, 25, 26, 42, 14, 22, 2, 18, 28, 1, 37, 21, 9, 36, 12, 39, 2, 12, 3, 9, 38, 20, 21, 21, 40, 31, 16, 31, 1, 30, 13, 36, 16, 43, 13, 29, 39, 20, 20, 16, 20, 24, 30, 40, 4, 35, 16, 21, 19, 8, 13, 41, 30, 24, 31, 46, 47, 13, 7, 4, 11, 45, 32, 27, 9, 7, 36, 26, 42, 15, 45, 39, 38, 23, 3, 1, 11, 29, 13, 41, 12, 2, 6, 19, 8, 49, 46, 46, 42, 9, 39, 14, 48, 34, 34, 35, 19, 25, 7, 23, 39, 7, 14, 14, 32, 38, 2, 34, 18, 17, 22, 29, 48, 45, 6, 10, 28, 41, 13, 2, 23, 49, 36, 33, 31, 28, 17, 25, 20, 2, 10, 24, 2, 21, 23, 29, 26, 49, 25, 10, 1, 8, 6, 38, 30, 12, 27, 15, 20, 37, 32, 25, 23, 48, 33, 45, 30, 37, 33, 23, 1, 4, 8, 45, 21, 29, 22, 23, 48, 30, 15, 43, 13, 16, 36, 39, 3, 28, 8, 8, 1, 29, 18, 6, 20, 47, 44, 8, 46, 11, 32, 47, 8, 46, 49, 30, 48, 48, 39, 27, 3, 48, 39, 33, 24, 19, 27, 8, 30, 25, 17, 20, 24, 30, 10, 4, 7, 6, 42, 42, 31, 24, 27, 16, 8, 33, 32, 38, 13, 26, 22, 43, 34, 11, 32, 31, 1, 28, 44, 8, 26, 44, 23, 38, 39, 10, 27, 37, 2, 3, 9, 42, 46, 4, 24, 6, 19, 23, 25, 23, 32, 34, 35, 6, 43, 29, 42, 41, 21, 19, 37, 23, 1, 45, 40, 49, 33, 19, 2, 49, 46, 3, 32, 7, 46, 31, 24, 44, 29, 14, 47, 1, 33, 16, 13, 6, 1, 4, 10, 39, 22, 28, 5, 1, 13, 28, 42, 24, 43, 5, 2, 13, 22, 10, 4, 32, 12, 46, 2, 7, 1, 38, 34, 37, 46, 18, 13, 34, 44, 23, 13, 2, 46, 33, 41, 15, 43, 30, 36, 24, 11, 4, 1, 46, 31, 13, 32, 12, 39, 39, 21, 12, 30, 23, 6, 47, 34, 35, 43, 23, 5, 31, 8, 28, 16, 47, 43, 22, 17, 7, 29, 39, 3, 23, 20, 46, 12, 20, 42, 49, 42, 26, 6, 1, 4, 16, 15, 13, 2, 13, 42, 39, 48, 15, 46, 48, 34, 19, 18, 29, 32, 7, 43, 4, 5, 28, 44, 31, 20, 46, 11, 1, 47, 37, 2, 34, 7, 12, 8, 28, 44, 45, 41, 19, 19, 33, 8, 42, 26, 34, 25, 49, 5, 43, 1, 27])
opt = [0, 1, 1, 1, 0]

#Fitness function

You can use whatever fitness function you please for optimising the Knapsack.
However the function bellow that calculates the total benefit will be the one used to evaluate your genotype solution.
If you wish to modify the fitness function for your GA implementtion make sure you define a new function so you can also use this one to evaluate your solutions at the end.

In [25]:
def fitness_function(genotype, benefits, volumes, max_volume):
  vol = 0
  ben = 0
  # iterate all nonzero elements
  for i in np.nonzero(genotype)[0]:  
      ben += benefits[i]
      vol += volumes[i]
  if vol > max_volume:
    return 0
  else:
    return ben

#GA Class Template

For the purpose of the challenge you must use the below template for your code.
The most important thing is that you use a class called `GA` and make sure it has a function called  `evolve()`.
This function will be called to evaluate your solution. You can off course change the arguments of the object constructor to accomodate your parameters such as mutate rate, recombination rate etc.


*   The `initalise_pop()` function can be used to change the method by which the population is initialised. 
Note here that the function uses the `self.pop_size` and `self.num_items` variables to generate the population matrix.
If you wish to change this function you can, but make sure the population is initalised during the construction of the object.
*   The `evolve()` function is the function that the evaluation code will call to evolve the population and return the best genotype at the end. 


In [26]:
class GA:
  def __init__(self, pop_size: int, generations: int):
    self.benefits = benefits
    self.volumes = volumes
    self.max_vol = max_volume
    self.pop_size = pop_size
    self.generations = generations
    self.num_items = len(self.benefits)
    assert len(self.benefits) == len(self.volumes), "The benefits and volumes arrays should be equal size"
    self.genotype_len = len(self.benefits)
    # initialise the population
    self.pop = self.initialise_pop()

  def initialise_pop(self):
    pop = np.random.choice([0, 1], (self.pop_size, self.num_items))
    return np.squeeze(pop)
  
  def evolve(self):
    '''
    The function that the assignemnt evaluation code is going to call.
    It takes no arguments! All the hyperparameter of the GA class should be in the constructor!
    returns: 1D numpy array cotaining 1s and 0s. I.e. [1 1 1 0 1 0 1 1 1 0]
    '''
    pass
    ######
    # Your code goes in here
    # here you can call any other function you like
    # you can also create more functions within this class
    # The main loop of you algorithm would probaly need to be in here
    # This function should return a single genotype
    ######
import random
import numpy as np

def __init__(self, mutation_rate, crossover_rate, tournament_size, num_items):
    self.mutation_rate = mutation_rate
    self.crossover_rate = crossover_rate
    self.tournament_size = tournament_size
    self.num_items = num_items
    self.population = self._create_population()
    self.best_fitnesses = []

def _create_population(self):
    return [[random.randint(0, 1) for _ in range(self.num_items)] for _ in range(self.tournament_size)]

def mutate(self, parent):
    return [gene if random.random() > self.mutation_rate else 1 - gene for gene in parent]

def crossover(self, parent1, parent2):
    if random.random() < self.crossover_rate:
        point = random.randint(0, self.num_items)
        return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]
    else:
        return parent1, parent2

def select(self):
    selected = random.sample(self.population, self.tournament_size)
    return max(selected, key=self.fitness)

def fitness(self, parent):
    return sum(parent)

def run(self, generations):
    for _ in range(generations):
        parent1 = self.select()
        parent2 = self.select()
        child1, child2 = self.crossover(parent1, parent2)
        self.population.append(self.mutate(child1))
        self.population.append(self.mutate(child2))
        self.population.sort(key=self.fitness, reverse=True)
        self.population = self.population[:self.tournament_size]
        self.best_fitnesses.append(self.fitness(self.population[0]))


#For example lets see how you would implement a random (hill)parent

In [27]:
class GA:
    def __init__(self, population, pop_size, generations):
      self.benefits = benefits
      self.volumes = volumes
      self.max_vol = max_volume
      self.pop_size = pop_size
      self.generations = generations
      self.population = population  
      assert len(self.benefits) == len(self.volumes), "The benefits and volumes arrays should be equal size"
      self.num_items = len(self.benefits)
      self.parent = self.initialise_pop()
      self.parent_fit = None
    
    #def initialise_pop(self):
      # In this case the population is only 1 hillparent
      #pop = np.random.choice([0, 1], (self.pop_size, self.num_items))
      #return np.squeeze(pop)

    def evolve(self):
      '''
      The function that the assignemnt evaluation code is going to call.
      It takes no arguments! All the hyperparameter of the GA class should be in the constructor!
      returns: 1D numpy array cotaining 1s and 0s. I.e. [1 1 1 0 1 0 1 1 1 0]
      '''
      self.parent_fit = fitness_function(self.parent, self.benefits, self.volumes, self.max_vol)
      #for every generation
      for i in range(self.generations):
          # copy the parent and mutate
          new_parent = self.mutate(copy.deepcopy(self.parent))
          self.parent = new_parent
          self.parent_fit = fitness_function(new_parent, self.benefits, self.volumes, self.max_vol)
      
      return self.parent

    def initialise_pop(self):
            parents = []
            for _ in range(self.population):
                parent = []
                for _ in range(self.num_items):
                    k = random.randint(0, 1)
                    parent.append(k)
                parents.append(parent)
            return parents

    def get_parent(self):
        return self.parent
    # set the details of this problem
    def properties(self, volumes, benefits, fitness, max_volume, population):
        self.volumes = volumes
        self.benefits = benefits
        self.fitness = fitness
        self.max_vol = max_volume
        self.population = population
        self.initialize()

    # calculate the fitness function
    def fitness(self, item):
            vol = 0
            ben = 0
            for index, i in enumerate(item):
                    if i == 0:
                            continue
                    else:
                            vol += self.volumes[i]
                            ben += self.benefits[i]
            if vol > self.max_vol:
                  return -1
            else:
                  return ben


    def evaluation(self):

        # loop through parents and calculate fitness
        best_pop = self.population // 2
        for i in range(len(self.parents)):
          parent = self.parents[i]
          ft = self.fitness(parent)
          self.bests.append((ft, parent))

        # sort the fitness list by fitness    
        self.bests.sort(key=operator.itemgetter(0), reverse=True)
        self.best_p = self.bests[:best_pop]
        self.best_p = [x[1] for x in self.best_p]

    # mutate children after certain condition
    def mutate(self, ch):

      for i in range(len(ch)):		
        k = random.uniform(0, 1)
        if k > 0.5:

          if ch[i] == 1:
            ch[i] = 0
          else: 
            ch[i] = 1
      return ch

    # crossover two parents to produce two children by mixing them under random ration each time
    def crossover(self, ch1, ch2):

        threshold = random.randint(1, len(ch1)-1)
        tmp1 = ch1[threshold:]
        tmp2 = ch2[threshold:]
        ch1 = ch1[:threshold]
        ch2 = ch2[:threshold]
        ch1.extend(tmp2)
        ch2.extend(tmp1)

        return ch1, ch2

    def run(self):

        # run the evaluation once
        self.evaluation()
        newparents = []
        pop = len(self.best_p)-1

        # create a list with unique random integers
        sample = random.sample(range(pop), pop)
        for i in range(0, pop):
          # select the random index of best children to randomize the process
          if i < pop-1:
            r1 = self.best_p[i]
            r2 = self.best_p[i+1]
            nchild1, nchild2 = self.crossover(r1, r2)
            newparents.append(nchild1)
            newparents.append(nchild2)
          else:
            r1 = self.best_p[i]
            r2 = self.best_p[0]
            nchild1, nchild2 = self.crossover(r1, r2)
            newparents.append(nchild1)
            newparents.append(nchild2)


        for i in range(len(newparents)):
          newparents[i] = self.mutate(newparents[i])

        if self.fitness in newparents:
          print ("optimal found in {} generations" .format(self.iterated))
        else:
          self.iterated += 1
          print("recreate generations for {} time" .format(self.iterated))
          self.parents = newparents
          self.bests = []
          self.best_p = []
          self.run()
          

#Submit your GA configuration and check that it works
In order to submit your code you must create a list of all your constructor arguments in the correct order. See the example bellow for the hillparent example code.

**Before setting your parameters and submitting your GA configuration make sure it runs under the 2 minutes constraint!!**

In [28]:
## Place your arguments in a list called args in the same order they are defined in the class constructor.
pop_size = 1
generations = 1000
population =100
# Notice here we place the arguments of the GA constructor in a list in the same order that they
# are defined in the GA class constructor
args = [pop_size, generations, population]

# The code below should run without problem if everything is set up correctly.
# Make sure to ask the TAs for help if this code is not running
ga = GA(*args)
parent = ga.get_parent()
print(parent)
best_parent = ga.get_parent()
print(best_parent, ga.run)


[[0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,

#Solution Evaluation

### Students should not modify these parameters!!
### Parameters may only be modfied by the Lecturer or TAs.

## Problem sampling

In [29]:
import time
# the problem generation function
def circ(no_of_items: int, R=50, d=2/3, max_vol_perc=.45):
  vol = np.random.randint(1, R, no_of_items)
  ben = d*np.sqrt(4*R**2-(vol-2*R)**2)
  max_vol = int(round(np.sum(vol)*max_vol_perc))
  return ben, vol, max_vol

## Standard fitness function

In [30]:
def fitness_function(genotype, benefits, volumes, max_volume):
  vol = 0
  ben = 0
  # iterate all nonzero elements
  for i in np.nonzero(genotype)[0]:  
      ben += benefits[i]
      vol += volumes[i]
  if vol > max_volume:
    return 0
  else:
    return ben

## Setting the evaluation parameters.

In [31]:
# Evaluation parameters
# The evaluations code uses these as global variables
# Do not change the signatures!
# seed to be selected by Lecturer
np.random.seed(None) 
number_of_items = 500
repeats = 3 # Number of evaluations
max_time = 120 # in seconds

# problem generator
benefits, volumes, max_volume = circ(number_of_items)

# log variables
time_complexities = []
best_gntps = []

In [32]:
for i in range(repeats):
  tic = time.perf_counter()
  ga = GA(*args)
  gntp = ga.evolve()
  toc = time.perf_counter()
  time_complexities.append(toc-tic)
  best_gntps.append(gntp)

mean_time = np.mean(time_complexities)
if mean_time > max_time:
  print(f'Took more than {max_time} seconds to run')
else:
  best_fits = []
  for g in best_gntps:
    best_fits.append(fitness_function(g, benefits, volumes, max_volume))
  mean_fitness = np.mean(best_fits)
  max_fitness = np.max(best_fits)
  print(f'Evaluation was repreated {repeats} times. Max fitness: {max_fitness}\
  Average fitness: {mean_fitness}, with average runtime of {mean_time} seconds.')

Evaluation was repreated 3 times. Max fitness: 22.744962812309304  Average fitness: 7.581654270769768, with average runtime of 0.003986806333349098 seconds.
